In [1]:
import numpy as np

# Sort proteins of STRING

In [2]:
f_name = '../data/PPInetwork/STRING/9606.protein.links.v11.5.txt'
f = open(f_name, 'r')
data = f.read()
rows = data.split('\n')[1:-1]

all_protein = [None]*2*len(rows)
for i, r in enumerate(rows):
    proteins = r.split()
    all_protein[i] = proteins[0]
    all_protein[-(i+1)] = proteins[1]
unipue_protein = list(set(all_protein))

In [3]:
unipue_protein[:10]

['9606.ENSP00000230658',
 '9606.ENSP00000266376',
 '9606.ENSP00000322276',
 '9606.ENSP00000246006',
 '9606.ENSP00000246957',
 '9606.ENSP00000322784',
 '9606.ENSP00000367746',
 '9606.ENSP00000310263',
 '9606.ENSP00000340648',
 '9606.ENSP00000379203']

# Scraping
webページがjavascriptで書かれててうまく情報がしゅとくできなかったので、selenuiumでスクレイピングする

In [28]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from webdriver_manager.chrome import  ChromeDriverManager
from tqdm import tqdm
import time

In [73]:
# settings
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(
        ChromeDriverManager().install(),
        options=options
        )

entries = {
    k:None
    for k in unipue_protein
}
for i, protein in enumerate(tqdm(unipue_protein, total=len(unipue_protein))):  
    try:
        url =  f'https://www.uniprot.org/uniprotkb?query={protein}'
        driver.get(url)
        time.sleep(0.5)
        # slect UIs
        if i == 0:
            time.sleep(2)
            driver.find_element_by_xpath('/html/body/form/div/span/label[2]/input').click()
            driver.find_element_by_xpath('/html/body/form/div/section/button').click()
        # analyze web page information
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        # get protein name
        items = soup.findAll(class_='BqBnJ')
        entries[protein] = [item.text for item in items]
    except Exception as e:
        pass
driver.close()

100%|██████████| 19385/19385 [4:47:31<00:00,  1.12it/s]  


In [77]:
import json

with open('../data/PPInetwork/STRING/STRINGId_UniProtId.json', 'w') as f:
    json.dump(entries,f)

# Download FASTA and GO annotations from Uniprot

In [78]:
import json
import requests

In [79]:
f_name = '../data/PPInetwork/STRING/STRINGId_UniProtId.json'
with open(f_name, 'r') as f:
    string_uniprot_dict = json.load(f)

In [80]:
all_uniprot_Ids = []
for value in string_uniprot_dict.values():
    for item in value:
        all_uniprot_Ids.append(item)
unique_uniprot_Ids = list(set(all_uniprot_Ids))

In [115]:
for i, entry in enumerate(tqdm(unique_uniprot_Ids, total = len(unique_uniprot_Ids))):
    if i >= 450:
        try:
            url_fasta = f'https://rest.uniprot.org/uniprotkb/{entry}.fasta'
            url_text = f'https://rest.uniprot.org/uniprotkb/{entry}.txt'
            response_fasta = requests.get(url_fasta)
            response_text = requests.get(url_text)
            if response_fasta.status_code == 200 and response_text.status_code== 200:
                fasta = response_fasta.content
                text = response_text.content
                with open(f'../data/PPInetwork/STRING/UniProt/fasta/{entry}.fasta', 'wb') as f:
                    f.write(fasta)
                with open(f'../data/PPInetwork/STRING/UniProt/text/{entry}.text', 'wb') as f:
                    f.write(text)
            else:
                print(i)
        except Exception as e:
            print(i, e)
            pass

  6%|▋         | 1385/21719 [23:22<32:25:29,  5.74s/it]

1384 HTTPSConnectionPool(host='rest.uniprot.org', port=443): Max retries exceeded with url: /uniprotkb/Q9UL52.txt (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1123)')))


  7%|▋         | 1504/21719 [26:49<49:26:18,  8.80s/it]

1503 ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


  7%|▋         | 1524/21719 [28:16<51:46:10,  9.23s/it]

...

 14%|█▍        | 3000/21719 [1:28:21<74:07:39, 14.26s/it] 

2999 HTTPSConnectionPool(host='rest.uniprot.org', port=443): Max retries exceeded with url: /uniprotkb/A8K9Q8.fasta (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1123)')))


 30%|██▉       | 6508/21719 [2:59:43<33:39:26,  7.97s/it]

6507 HTTPSConnectionPool(host='rest.uniprot.org', port=443): Max retries exceeded with url: /uniprotkb/Q8N4V1.txt (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1123)')))


100%|██████████| 21719/21719 [9:15:21<00:00,  1.53s/it]  
